In [1]:
%matplotlib inline
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta

# Importing the datasets 

In [2]:
training_file = "train.p"
testing_file = "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

## Dataset facts 

In [3]:
n_train = len(y_train)
n_test = len(y_test)
image_shape  = np.shape(X_train[1])
n_classes = len(np.unique(y_train))

img_size_flat = image_shape[0] * image_shape[1]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

print("Image Size Flat =", img_size_flat)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43
Image Size Flat = 1024


# One-hot encoding the labels

In [4]:
y_train_encoded = np.eye(43)[y_train]
y_test_encoded = np.eye(43)[y_test]

In [5]:
image_shape[0]

32

# Pre-processing the images


# Batching

In [6]:
def random_batch():
    # Number of images in the training-set.
    num_images = len(X_train)

    # Create a random index.
    idx = np.random.choice(num_images,
                           size=batch_size,
                           replace=False)

    # Use the random index to select random images and labels.
    features_batch = X_train[idx, :]
    labels_batch = y_train_encoded[idx, :]

    return features_batch, labels_batch

# Setting up the TensorFlow model

## setting up the constants

In [7]:
x = tf.placeholder(tf.float32, [None, 32,32,3])
y = tf.placeholder(tf.float32, [None, n_classes])

# sets up a place for the non one-hot classes to go
y_true_cls = tf.placeholder(tf.int64, [None])

In [8]:
x_flat = tf.reshape(x, [-1, img_size_flat*3])
x_flat


<tf.Tensor 'Reshape:0' shape=(?, 3072) dtype=float32>

## setting up the variables (the things that change -- weights and biases)

In [9]:
weights = tf.Variable(tf.zeros([img_size_flat*3, n_classes]))
biases = tf.Variable(tf.zeros([n_classes]))


## model hyperparameters

In [20]:
learning_rate = 0.1
# n_iterations = 1
batch_size = 10
display_step = 100


# TensorFlow model

In [11]:
logits = tf.matmul(x_flat, weights) + biases
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, dimension = 1)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels= y)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





# TensorFlow Session


In [12]:
def optimize(n_iterations):
      for iteration in range(n_iterations):
        features_batch, labels_true_batch = random_batch()
        feed_dict_batch = {x: features_batch, y: labels_true_batch}
        sess.run(optimizer, feed_dict = feed_dict_batch)

In [21]:
start_time = time.time()

sess = tf.Session()
sess.run(tf.initialize_all_variables())

optimize(10000)    

end_time = time.time()
compute_time = end_time - start_time

print("compute time: " , str(timedelta(seconds = int(round(compute_time)))))



compute time:  0:00:19


In [22]:
feed_dict_test = {x: X_test,
                  y: y_test_encoded,
                  y_true_cls:y_test}

In [23]:
acc = sess.run(accuracy,feed_dict=feed_dict_test)
print(acc)
    

0.628424


In [17]:
img = X_test[0:5]
# my_img_flat = np.reshape(img, 3072)
# feed_dict_test = {x: my_img_flat}
classification = sess.run(y, feed_dict)
(my_img_flat)

NameError: name 'feed_dict' is not defined


# Predicting the class for one image

helpful source: http://stackoverflow.com/questions/33711556/making-predictions-with-a-tensorflow-model

In [18]:
X_test[0]

array([[[116, 139, 174],
        [116, 137, 171],
        [118, 138, 172],
        ..., 
        [ 98, 114, 143],
        [ 97, 121, 147],
        [ 85, 105, 130]],

       [[115, 142, 176],
        [115, 140, 174],
        [117, 141, 175],
        ..., 
        [119, 143, 176],
        [120, 141, 173],
        [120, 139, 171]],

       [[117, 141, 174],
        [117, 142, 175],
        [114, 140, 172],
        ..., 
        [120, 144, 179],
        [122, 144, 179],
        [120, 143, 177]],

       ..., 
       [[117, 137, 167],
        [115, 134, 164],
        [117, 135, 164],
        ..., 
        [115, 137, 169],
        [116, 136, 168],
        [118, 139, 172]],

       [[116, 136, 166],
        [114, 134, 168],
        [113, 131, 166],
        ..., 
        [113, 135, 167],
        [117, 136, 165],
        [115, 139, 167]],

       [[112, 135, 167],
        [110, 134, 165],
        [119, 135, 165],
        ..., 
        [115, 138, 167],
        [117, 138, 167],
        [114, 140,

In [19]:
feed_dict = {x: X_test[10000:10005]}
classification = sess.run(y_pred, feed_dict)
cls = np.argmax(classification, axis=1)
print(cls)

[27 12  2 38 25]
